# Network Testing

## Includes

In [ ]:
# mass includes
import os, sys, warnings
import torch as t
from pickle import dump
from tqdm.notebook import tqdm

# add paths for all sub-folders
paths = [root for root, _, _ in os.walk('.')\
         if 'evals' not in root]
for item in paths:
    sys.path.append(item)

from ipynb.fs.full.config import Config
from ipynb.fs.full.network import *
from ipynb.fs.full.dataLoader import *
from ipynb.fs.full.util import *

## Initialization

In [ ]:
# define models
opt = Config()
net_enc = Encoder().to(opt.device)
net_sum = Summarizer().to(opt.device)

# load pre-trained weights
net_enc.load()
net_sum.load()

# dataset for testing
test_dataset = ImageSet(opt, mode='test')
test_loader = t.utils.data.DataLoader(test_dataset)

# make folders
feature_path = os.path.join(opt.save_path, 'features')
if os.path.exists(feature_path) is False:
    os.makedirs(feature_path)

## Testing entry

In [ ]:
# set to evaluation mode
net_enc.eval()
net_sum.eval()

for (img, f_name) in tqdm(test_loader, desc='progress',
                          total=len(test_loader)):
    # copy to device
    img = img.to(opt.device)

    with t.no_grad():
        # extract local encodings
        pred_loc_feats = net_enc(img)

        # extract global encodings
        pred_glb_feats = net_sum(pred_loc_feats)

    # save predictions to file
    save_dict = {
        'local_feats': pred_loc_feats.cpu().numpy(),
        'global_feats': pred_glb_feats.cpu().numpy()
    }
    file_path = os.path.join(feature_path, f_name[0] + '.pkl')
    with open(file_path, 'wb') as file:
        dump(save_dict, file)